# Prototype Evals
This notebook shows the use of the dummy feedback function provider which
behaves like the huggingface provider except it does not actually perform any
network calls and just produces constant results. It can be used to prototype
feedback function wiring for your apps before invoking potentially slow (to
run/to load) feedback functions.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/quickstart/prototype_evals.ipynb)

## Setup

### Install dependencies
Let's install some of the dependencies for this notebook if we don't have them already

In [ ]:
#! pip install trulens-eval==0.17.0b

In [ ]:
import threading

from examples.frameworks.custom.custom_app import CustomApp

from trulens_eval import Feedback
from trulens_eval import Tru
from trulens_eval.feedback.provider.hugs import Dummy
from trulens_eval.tru_custom_app import TruCustomApp
from trulens_eval.utils.threading import TP

tru = Tru()

tru.reset_database()

tru.start_dashboard()

In [ ]:
# hugs = Huggingface()
hugs = Dummy()

f_positive_sentiment = Feedback(hugs.positive_sentiment).on_output()

In [ ]:
# Create custom app:
ca = CustomApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
    # feedback_mode=FeedbackMode.WITH_APP
    feedbacks=[f_positive_sentiment]
)

In [ ]:
with ta:
    for i, q in enumerate(["hello there"] * 100):
        # Track number of requests, number of threads, and number of promises to fulfull
        print(f"\rrequest {i} ", end="")
        print(f"thread count={threading.active_count()}, promises={TP().promises.qsize()}", end="")

        res = ca.respond_to_query(input=q)